<a href="https://colab.research.google.com/github/patryktk/175IC-machine-learning-21176/blob/main/Lab5/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy --upgrade
!python -m spacy download pl_core_news_sm
!pip install textacy
import spacy
import pl_core_news_sm
import re
from spacy.tokenizer import Tokenizer
nlp = pl_core_news_sm.load()


In [ ]:
#String Read
about_text = ('Tutorial spacy do laboratrium, który jest dość długi, żeby sprawdzić inforacje')
about_doc = nlp(about_text)
print([token.text for token in about_doc])

In [ ]:
#Text file read
file_name = 'Notatnik.txt'
file_text = open(file_name).read()
file_doc = nlp(file_text)
#Extract token
print([token.text for token in file_doc])

In [ ]:
sentences =list(about_doc.sents)
len(sentences)
for sentences in sentences:
  print(sentences)

In [ ]:
def kropki(doc): #ta funkcja powoduje że w momencie wyczytania w tokenie '...' przenosimy tekst do kolejnej lini
  for token in doc[:-1]:
    if token.text == '...':
      doc[token.i+1].is_sent_start = True
  return doc

text_kropki = ('To jest tekst ... zaweierajacy'
              'duza ilosc ... kropek.')
custom_nlp = pl_core_news_sm.load() #ładujemy nasz język
custom_nlp.add_pipe(kropki, before='parser') #edytujemy nasz tekst i zwracamy tekest przed parserem
custom_ell_doc = custom_nlp(text_kropki) #ładujemy nasz tekst do zmiennej i 
custom_ell_sent = list(custom_ell_doc.sents) #elementy łączymy w liste, aby tekst był spójny
for sentences in custom_ell_sent:
  print(sentences)

In [ ]:
#Tokenization

for token in about_doc:
  print (token, token.idx, token.text_with_ws,
        token.is_alpha, token.is_punct, token.is_space,
         #is_alpha - sprawdza czy znaki są z alfabetu
         #is_punct - sprawdza znaki interpunkcyjne
         #is_space szuka spacji
         #shape - kształt słowa, rozmiar liter
         #is_stop-  sprawdza czy to stop word czy nie
        token.shape_, token.is_stop)

In [ ]:
#Tworzenie własnego tokenu
prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')
def customize_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,suffix_search=suffix_re.search,infix_finditer=infix_re.finditer,token_match=None)
custom_token_text = 'To-jest-mój-tokenizer'
nlp.tokenizer = customize_tokenizer(nlp)
custom_tokenizer_doc = nlp(custom_token_text)
print([token.text for token in custom_tokenizer_doc])

In [ ]:
spacy_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
len(spacy_stopwords)

for stop_word in list(spacy_stopwords)[:10]:
  print(stop_word)

#usuwanie stop wordow
for token in about_doc:
  if not token.is_stop:
    print (token)

In [ ]:
# sprowadzanie słów do form podstawowych
lemmatization_text = ('To jest jak przykładowy przykład przykładu działania przykładu')
lemmatization_doc = nlp(lemmatization_text)
for token in lemmatization_doc:
  print(token, token.lemma_)

In [ ]:
#Częstotliwość występownaia słów w tekście
from collections import Counter
complete_text=(about_text)
complete_doc = nlp(about_text)

words = [token.text for token in complete_doc
         if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

#Najczęstsze słowa
common_words = word_freq.most_common(5)
print (common_words)

#Słowa unikatowe
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

#Ropoznanie części mowy
for token in complete_doc:
  print(token, token.tag_, token.pos_, spacy.explain(token.tag_),sep=', ')

In [39]:
#Kategoryzacja
nouns = []
adjectives = []
for token in about_doc:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'ADJ':
        adjectives.append(token)

print(f'Rzeczowniki: {nouns}')
print(f'Przymiotniki: {adjectives}')

Rzeczowniki: [Tutorial, spacy, laboratrium, inforacje]
Przymiotniki: [który, długi]


In [ ]:
#Wizualizacja
from spacy import displacy
about_displaCy_text = ('Laborka robiona na ostatnia chwile')
about_displaCy_doc = nlp(about_displaCy_text)
displacy.render(about_displaCy_doc, style='dep', jupyter=True)
#Aby wyświetlić w jupiter notebooku, z którego korzystam używaym ww. komendy

In [ ]:
#przetwarzanie tekstu, na format łatwy do analizy przez NLP
def is_token_allowed(token):
    if not token or not token.string.strip() or token.is_stop or token.is_punct:
        return False
    return True

def preprocess_token(token):
    return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)for token in complete_doc if is_token_allowed(token)]
print(complete_filtered_tokens)

In [ ]:
#Wyciąganie imienia i nazwisko, ale niestety nie działa
from spacy.matcher import Matcher
def extract_full_name(nlp_doc):
    pattern = [{'POS': 'PROPN '}, {'POS': 'PROPN'}]
    matcher.add('FULL_NAME', None, pattern)
    matches = matcher(nlp_doc)
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text

new_text_extract="Patryk Tkaczyk jedzenie to jest test Test"
new_text_extract_doc=nlp(new_text_extract)
matcher = Matcher(nlp.vocab)
print(extract_full_name(new_text_extract_doc))

new_text_extract_eg="Gus Proto"
new_text_extract_doc_eg=nlp(new_text_extract_eg)
matcher = Matcher(nlp.vocab)
print(extract_full_name(new_text_extract_doc_eg))

In [ ]:
#Wyciąganie numeru telefonu
phone_number=("To mój numer: 666 668 152")

def extract_phone_number(nlp_doc):
	pattern = [{"SHAPE": "ddd"}, {"SHAPE": "ddd"},{"SHAPE": "ddd"}]
	matcher.add('PHONE_NUMBER', None, pattern)
	matches = matcher(nlp_doc)
	for match_id, start, end in matches:
		span = nlp_doc[start:end]
		return span.text

conference_org_doc = nlp(phone_number)
extract_phone_number(conference_org_doc)

In [ ]:
#Strukutra zdania:
text = ('Przykładowy tekest to tekst jest')
text_doc = nlp(text)
for token in text_doc:
  print (token.text, token.tag_, token.head.text, token.dep_)
displacy.render(text_doc, style='dep', jupyter=True)

In [ ]:
#Nawigacja do drzew
one_line_about_text = ('Tańczymy jakby jutro miało nie być, myslovitz')
one_line_about_doc = nlp(one_line_about_text)

#Podzielnie 3 słowa oraz wyświetlenie sąsiadów słowa
print([token.text for token in one_line_about_doc[3].children])
print (one_line_about_doc[3].nbor(-1))
print (one_line_about_doc[3].nbor())
#Poddrzewao słowa 'jutro'
print (list(one_line_about_doc[3].subtree))

In [ ]:
#Wykrywanie czasowników
import textacy
about_talk_text=('Grupa kolegów chodziła całą noc, po osiedlu i rzucali kamieniami w okna.')
pattern = [{"POS": "VERB", "OP": "*"},{"POS": "ADV", "OP": "*"},{"POS": "VERB", "OP": "+"},{"POS": "PART", "OP": "*"}]
about_talk_doc = textacy.make_spacy_doc(about_talk_text, nlp)
verb_phrases = textacy.extract.matches(about_talk_doc, pattern)
for chunk in verb_phrases:
    print(chunk.text)

for chunk in about_talk_doc.noun_chunks:
    print (chunk)

In [ ]:
#Używanie NER do wyszukiwania konkretnych danych i ukrywania ich

survey_text =('Jan Nowak nie wie co się dzieje natomiast Anna Dymna doskonale zdaje sobie sprawę, że Tomasz Kos zjadł jej czekoaldki')
def replace_person_names(token):
    if token.ent_iob != 0 and token.ent_type_ == 'persName':
        return '[Zgadnij] '
    return token.string

def redact_names(nlp_doc):
    for ent in nlp_doc.ents:
        ent.merge()
    tokens = map(replace_person_names, nlp_doc)
    return ''.join(tokens)

survey_doc = nlp(survey_text)
print(redact_names(survey_doc))
print(survey_text)